In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Function definitions for evaluation metrics
def calculate_mse(actual, predicted):
    return np.mean((actual - predicted) ** 2)

def calculate_rmse(actual, predicted):
    return np.sqrt(calculate_mse(actual, predicted))

def calculate_mae(actual, predicted):
    return np.mean(np.abs(actual - predicted))

# File paths (adjust these paths as needed)
file_path = 'AirborneEmissions_Processed.xlsx'  # Input Excel file path
output_file_path = 'predictions_mc_with_metrics.xlsx'  # Output Excel file
images_output_dir = 'simulation_images'  # Directory to save images

os.makedirs(images_output_dir, exist_ok=True)

# Load the Excel file
excel_file = pd.ExcelFile(file_path)

num_simulations = 1000
future_years = np.arange(2023, 2031)

all_simulation_results = {}

# DataFrame to store evaluation metrics
metrics_list = []  # Use a list to collect metrics

for sheet_name in excel_file.sheet_names:
    sheet_df = pd.read_excel(file_path, sheet_name=sheet_name)
    print(f"Processing sheet: {sheet_name}, Number of Columns: {sheet_df.shape[1]}")
    
    years = sheet_df.columns[1:]  # Assuming first column is element names
    elements = sheet_df.iloc[:, 0]  # Element names
    
    # Calculate percentage changes for historical data
    yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)
    
    simulation_results = pd.DataFrame()
    
    for element in elements:
        historical_emissions = sheet_df.loc[sheet_df.iloc[:, 0] == element, years].values.flatten()
        historical_years = years.astype(int)
        
        # Remove NaN values from historical emissions and years
        valid_indices = ~np.isnan(historical_emissions)
        historical_emissions = historical_emissions[valid_indices]
        historical_years = historical_years[valid_indices]
        
        # Recalculate years and percentage changes for valid data
        years_valid = years[valid_indices]
        yearly_change_valid = yearly_change[years_valid].loc[sheet_df.iloc[:, 0] == element].values.flatten()
        yearly_change_valid = yearly_change_valid[~np.isnan(yearly_change_valid)]  # Remove NaNs
        
        mean_change = np.mean(yearly_change_valid)
        std_change = np.std(yearly_change_valid)
        
        # Total number of years to simulate (historical + future)
        total_years = len(historical_years) + len(future_years)
        
        all_simulations = np.zeros((num_simulations, total_years))
        
        for sim in range(num_simulations):
            simulated_emissions = [historical_emissions[-1]]  # Start from the last historical value
            
            for _ in future_years:
                random_change = np.random.normal(mean_change, std_change)
                new_value = simulated_emissions[-1] * (1 + random_change)
                simulated_emissions.append(new_value)
            
            # Combine historical emissions and future simulated emissions
            total_emissions = np.concatenate([historical_emissions, simulated_emissions[1:]])
            all_simulations[sim, :] = total_emissions
        
        # Calculate mean predictions across simulations
        mean_emissions = np.mean(all_simulations, axis=0)
        
        # Calculate evaluation metrics for the historical period
        num_historical_years = len(historical_emissions)
        mse = calculate_mse(historical_emissions, mean_emissions[:num_historical_years])
        rmse = calculate_rmse(historical_emissions, mean_emissions[:num_historical_years])
        mae = calculate_mae(historical_emissions, mean_emissions[:num_historical_years])
        
        # Store metrics in a list
        metrics_list.append({
            'Sheet': sheet_name,
            'Element': element,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae
        })
        
        # Store mean emissions in simulation results
        simulation_results[element] = mean_emissions
        
        # Calculate confidence intervals
        lower_bound = np.percentile(all_simulations, 5, axis=0)
        upper_bound = np.percentile(all_simulations, 95, axis=0)
        
        # Prepare years for plotting (should match the number of emissions)
        all_years = np.concatenate([historical_years, future_years])
        
        # Ensure all_years and mean_emissions have the same length before plotting
        if len(all_years) == len(mean_emissions):
            # Plotting mean emissions with confidence intervals
            plt.figure(figsize=(12, 6))
            plt.plot(all_years, mean_emissions, label=f"Mean {element}")
            plt.fill_between(all_years, lower_bound, upper_bound, color='gray', alpha=0.3, label="90% Confidence Interval")
            
            # Plot actual historical emissions
            plt.scatter(historical_years, historical_emissions, color='red', label='Actual Historical Emissions')
            
            plt.xlabel("Year")
            plt.ylabel("Emissions")
            plt.title(f"Monte Carlo Simulation for {element} ({sheet_name})")
            plt.legend()
            plt.grid(True)
            
            image_file_path = os.path.join(images_output_dir, f"{sheet_name}_{element}_simulation.png")
            plt.savefig(image_file_path)
            plt.close()
            print(f"Graph saved for element '{element}' in sheet '{sheet_name}' at {image_file_path}")
        else:
            print(f"Skipping plot for {element} due to mismatch in years and emissions dimensions.")
    
    # Save simulation results for this sheet
    all_years = np.concatenate([historical_years, future_years])
    simulation_results.index = all_years
    all_simulation_results[sheet_name] = simulation_results

# Convert metrics list to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Save all simulation results and metrics to an Excel file
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, simulation_result in all_simulation_results.items():
        simulation_result.to_excel(writer, sheet_name=sheet_name)
    # Save metrics to a separate sheet
    metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

print(f"Monte Carlo simulation results and metrics saved to {output_file_path}")

Processing sheet: Arsenic, Number of Columns: 34


C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\2691852690.py:42: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'A' in sheet 'Arsenic' at simulation_images\Arsenic_A_simulation.png
Graph saved for element 'B' in sheet 'Arsenic' at simulation_images\Arsenic_B_simulation.png
Graph saved for element 'C' in sheet 'Arsenic' at simulation_images\Arsenic_C_simulation.png
Graph saved for element 'D' in sheet 'Arsenic' at simulation_images\Arsenic_D_simulation.png
Graph saved for element 'E' in sheet 'Arsenic' at simulation_images\Arsenic_E_simulation.png
Graph saved for element 'F' in sheet 'Arsenic' at simulation_images\Arsenic_F_simulation.png
Graph saved for element 'G' in sheet 'Arsenic' at simulation_images\Arsenic_G_simulation.png
Graph saved for element 'H' in sheet 'Arsenic' at simulation_images\Arsenic_H_simulation.png
Graph saved for element 'I' in sheet 'Arsenic' at simulation_images\Arsenic_I_simulation.png
Graph saved for element 'J' in sheet 'Arsenic' at simulation_images\Arsenic_J_simulation.png
Graph saved for element 'K' in sheet 'Arsenic' at simulation_images\Ar

C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\2691852690.py:42: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'B' in sheet 'Cadmium' at simulation_images\Cadmium_B_simulation.png
Graph saved for element 'C' in sheet 'Cadmium' at simulation_images\Cadmium_C_simulation.png
Graph saved for element 'D' in sheet 'Cadmium' at simulation_images\Cadmium_D_simulation.png
Graph saved for element 'E' in sheet 'Cadmium' at simulation_images\Cadmium_E_simulation.png
Graph saved for element 'F' in sheet 'Cadmium' at simulation_images\Cadmium_F_simulation.png
Graph saved for element 'G' in sheet 'Cadmium' at simulation_images\Cadmium_G_simulation.png
Graph saved for element 'H' in sheet 'Cadmium' at simulation_images\Cadmium_H_simulation.png
Graph saved for element 'I' in sheet 'Cadmium' at simulation_images\Cadmium_I_simulation.png
Graph saved for element 'J' in sheet 'Cadmium' at simulation_images\Cadmium_J_simulation.png
Graph saved for element 'K' in sheet 'Cadmium' at simulation_images\Cadmium_K_simulation.png
Graph saved for element 'L' in sheet 'Cadmium' at simulation_images\Ca

...

Graph saved for element 'A' in sheet 'Selenium' at simulation_images\Selenium_A_simulation.png
Graph saved for element 'B' in sheet 'Selenium' at simulation_images\Selenium_B_simulation.png
Graph saved for element 'C' in sheet 'Selenium' at simulation_images\Selenium_C_simulation.png
Graph saved for element 'D' in sheet 'Selenium' at simulation_images\Selenium_D_simulation.png
Graph saved for element 'E' in sheet 'Selenium' at simulation_images\Selenium_E_simulation.png
Graph saved for element 'F' in sheet 'Selenium' at simulation_images\Selenium_F_simulation.png
Graph saved for element 'G' in sheet 'Selenium' at simulation_images\Selenium_G_simulation.png
Graph saved for element 'H' in sheet 'Selenium' at simulation_images\Selenium_H_simulation.png
Graph saved for element 'I' in sheet 'Selenium' at simulation_images\Selenium_I_simulation.png
Graph saved for element 'J' in sheet 'Selenium' at simulation_images\Selenium_J_simulation.png
Graph saved for element 'K' in sheet 'Selenium' at

C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\2691852690.py:42: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'B' in sheet 'Vanadium' at simulation_images\Vanadium_B_simulation.png
Graph saved for element 'C' in sheet 'Vanadium' at simulation_images\Vanadium_C_simulation.png
Graph saved for element 'D' in sheet 'Vanadium' at simulation_images\Vanadium_D_simulation.png
Graph saved for element 'E' in sheet 'Vanadium' at simulation_images\Vanadium_E_simulation.png
Graph saved for element 'F' in sheet 'Vanadium' at simulation_images\Vanadium_F_simulation.png
Graph saved for element 'G' in sheet 'Vanadium' at simulation_images\Vanadium_G_simulation.png
Graph saved for element 'H' in sheet 'Vanadium' at simulation_images\Vanadium_H_simulation.png
Graph saved for element 'I' in sheet 'Vanadium' at simulation_images\Vanadium_I_simulation.png
Graph saved for element 'J' in sheet 'Vanadium' at simulation_images\Vanadium_J_simulation.png
Graph saved for element 'K' in sheet 'Vanadium' at simulation_images\Vanadium_K_simulation.png
Graph saved for element 'L' in sheet 'Vanadium' at

C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\2691852690.py:42: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'A' in sheet 'Zinc' at simulation_images\Zinc_A_simulation.png
Graph saved for element 'B' in sheet 'Zinc' at simulation_images\Zinc_B_simulation.png
Graph saved for element 'C' in sheet 'Zinc' at simulation_images\Zinc_C_simulation.png
Graph saved for element 'D' in sheet 'Zinc' at simulation_images\Zinc_D_simulation.png
Graph saved for element 'E' in sheet 'Zinc' at simulation_images\Zinc_E_simulation.png
Graph saved for element 'F' in sheet 'Zinc' at simulation_images\Zinc_F_simulation.png
Graph saved for element 'G' in sheet 'Zinc' at simulation_images\Zinc_G_simulation.png
Graph saved for element 'H' in sheet 'Zinc' at simulation_images\Zinc_H_simulation.png
Graph saved for element 'I' in sheet 'Zinc' at simulation_images\Zinc_I_simulation.png
Graph saved for element 'J' in sheet 'Zinc' at simulation_images\Zinc_J_simulation.png
Graph saved for element 'K' in sheet 'Zinc' at simulation_images\Zinc_K_simulation.png
Graph saved for element 'L' in sheet 'Zinc'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Function definitions for evaluation metrics
def calculate_mse(actual, predicted):
    return np.mean((actual - predicted) ** 2)

def calculate_rmse(actual, predicted):
    return np.sqrt(calculate_mse(actual, predicted))

def calculate_mae(actual, predicted):
    return np.mean(np.abs(actual - predicted))

# File paths (adjust these paths as needed)
file_path = 'AirborneEmissions_Processed.xlsx'  # Input Excel file path
output_file_path = 'predictions_mc_with_metrics.xlsx'  # Output Excel file
images_output_dir = 'simulation_images'  # Directory to save images

os.makedirs(images_output_dir, exist_ok=True)

# Load the Excel file
excel_file = pd.ExcelFile(file_path)

num_simulations = 1000
future_years = np.arange(2023, 2031)

all_simulation_results = {}

# DataFrame to store evaluation metrics
metrics_list = []  # Use a list to collect metrics

# Initialize dictionaries to hold metrics for each type
mse_summary = {}
rmse_summary = {}
mae_summary = {}

for sheet_name in excel_file.sheet_names:
    sheet_df = pd.read_excel(file_path, sheet_name=sheet_name)
    print(f"Processing sheet: {sheet_name}, Number of Columns: {sheet_df.shape[1]}")
    
    years = sheet_df.columns[1:]  # Assuming first column is element names
    elements = sheet_df.iloc[:, 0]  # Element names
    
    # Calculate percentage changes for historical data
    yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)
    
    simulation_results = pd.DataFrame()
    
    for element in elements:
        historical_emissions = sheet_df.loc[sheet_df.iloc[:, 0] == element, years].values.flatten()
        historical_years = years.astype(int)
        
        # Remove NaN values from historical emissions and years
        valid_indices = ~np.isnan(historical_emissions)
        historical_emissions = historical_emissions[valid_indices]
        historical_years = historical_years[valid_indices]
        
        # Recalculate years and percentage changes for valid data
        years_valid = years[valid_indices]
        yearly_change_valid = yearly_change[years_valid].loc[sheet_df.iloc[:, 0] == element].values.flatten()
        yearly_change_valid = yearly_change_valid[~np.isnan(yearly_change_valid)]  # Remove NaNs
        
        mean_change = np.mean(yearly_change_valid)
        std_change = np.std(yearly_change_valid)
        
        # Total number of years to simulate (historical + future)
        total_years = len(historical_years) + len(future_years)
        
        all_simulations = np.zeros((num_simulations, total_years))
        
        for sim in range(num_simulations):
            simulated_emissions = [historical_emissions[-1]]  # Start from the last historical value
            
            for _ in future_years:
                random_change = np.random.normal(mean_change, std_change)
                new_value = simulated_emissions[-1] * (1 + random_change)
                simulated_emissions.append(new_value)
            
            # Combine historical emissions and future simulated emissions
            total_emissions = np.concatenate([historical_emissions, simulated_emissions[1:]])
            all_simulations[sim, :] = total_emissions
        
        # Calculate mean predictions across simulations
        mean_emissions = np.mean(all_simulations, axis=0)
        
        # Calculate evaluation metrics for the historical period
        num_historical_years = len(historical_emissions)
        mse = calculate_mse(historical_emissions, mean_emissions[:num_historical_years])
        rmse = calculate_rmse(historical_emissions, mean_emissions[:num_historical_years])
        mae = calculate_mae(historical_emissions, mean_emissions[:num_historical_years])
        
        # Store metrics in a list
        metrics_list.append({
            'Sheet': sheet_name,
            'Element': element,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae
        })
        
        # Store metrics in summaries
        if sheet_name not in mse_summary:
            mse_summary[sheet_name] = []
        if sheet_name not in rmse_summary:
            rmse_summary[sheet_name] = []
        if sheet_name not in mae_summary:
            mae_summary[sheet_name] = []
        
        mse_summary[sheet_name].append(mse)
        rmse_summary[sheet_name].append(rmse)
        mae_summary[sheet_name].append(mae)

        # Store mean emissions in simulation results
        simulation_results[element] = mean_emissions
        
        # Calculate confidence intervals
        lower_bound = np.percentile(all_simulations, 5, axis=0)
        upper_bound = np.percentile(all_simulations, 95, axis=0)
        
        # Prepare years for plotting (should match the number of emissions)
        all_years = np.concatenate([historical_years, future_years])
        
        # Ensure all_years and mean_emissions have the same length before plotting
        if len(all_years) == len(mean_emissions):
            # Plotting mean emissions with confidence intervals
            plt.figure(figsize=(12, 6))
            plt.plot(all_years, mean_emissions, label=f"Mean {element}")
            plt.fill_between(all_years, lower_bound, upper_bound, color='gray', alpha=0.3, label="90% Confidence Interval")
            
            # Plot actual historical emissions
            plt.scatter(historical_years, historical_emissions, color='red', label='Actual Historical Emissions')
            
            plt.xlabel("Year")
            plt.ylabel("Emissions")
            plt.title(f"Monte Carlo Simulation for {element} ({sheet_name})")
            plt.legend()
            plt.grid(True)
            
            image_file_path = os.path.join(images_output_dir, f"{sheet_name}_{element}_simulation.png")
            plt.savefig(image_file_path)
            plt.close()
            print(f"Graph saved for element '{element}' in sheet '{sheet_name}' at {image_file_path}")
        else:
            print(f"Skipping plot for {element} due to mismatch in years and emissions dimensions.")
    
    # Save simulation results for this sheet
    all_years = np.concatenate([historical_years, future_years])
    simulation_results.index = all_years
    all_simulation_results[sheet_name] = simulation_results

# Convert metrics list to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Convert summaries to DataFrames for plotting
mse_summary_df = pd.DataFrame.from_dict(mse_summary, orient='index').transpose()
rmse_summary_df = pd.DataFrame.from_dict(rmse_summary, orient='index').transpose()
mae_summary_df = pd.DataFrame.from_dict(mae_summary, orient='index').transpose()

# Save all simulation results and metrics to an Excel file
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, simulation_result in all_simulation_results.items():
        simulation_result.to_excel(writer, sheet_name=sheet_name)
    # Save metrics to a separate sheet
    metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

print(f"Monte Carlo simulation results and metrics saved to {output_file_path}")

# Plot the summary metrics
plt.figure(figsize=(12, 8))

# Plot MSE
plt.subplot(3, 1, 1)
mse_summary_df.plot(kind='box', ax=plt.gca())
plt.title('Mean Squared Error (MSE) Summary')
plt.ylabel('MSE')

# Plot RMSE
plt.subplot(3, 1, 2)
rmse_summary_df.plot(kind='box', ax=plt.gca())
plt.title('Root Mean Squared Error (RMSE) Summary')
plt.ylabel('RMSE')

# Plot MAE
plt.subplot(3, 1, 3)
mae_summary_df.plot(kind='box', ax=plt.gca())
plt.title('Mean Absolute Error (MAE) Summary')
plt.ylabel('MAE')

plt.tight_layout()
plt.savefig('summary_metrics_box_plot.png')
plt.close()

print("Summary metrics plots saved as 'summary_metrics_box_plot.png'")

Processing sheet: Arsenic, Number of Columns: 34
Graph saved for element 'A' in sheet 'Arsenic' at simulation_images\Arsenic_A_simulation.png


C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\938653352.py:47: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'B' in sheet 'Arsenic' at simulation_images\Arsenic_B_simulation.png
Graph saved for element 'C' in sheet 'Arsenic' at simulation_images\Arsenic_C_simulation.png
Graph saved for element 'D' in sheet 'Arsenic' at simulation_images\Arsenic_D_simulation.png
Graph saved for element 'E' in sheet 'Arsenic' at simulation_images\Arsenic_E_simulation.png
Graph saved for element 'F' in sheet 'Arsenic' at simulation_images\Arsenic_F_simulation.png
Graph saved for element 'G' in sheet 'Arsenic' at simulation_images\Arsenic_G_simulation.png
Graph saved for element 'H' in sheet 'Arsenic' at simulation_images\Arsenic_H_simulation.png
Graph saved for element 'I' in sheet 'Arsenic' at simulation_images\Arsenic_I_simulation.png
Graph saved for element 'J' in sheet 'Arsenic' at simulation_images\Arsenic_J_simulation.png
Graph saved for element 'K' in sheet 'Arsenic' at simulation_images\Arsenic_K_simulation.png
Graph saved for element 'L' in sheet 'Arsenic' at simulation_images\Ar

C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\938653352.py:47: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'B' in sheet 'Cadmium' at simulation_images\Cadmium_B_simulation.png
Graph saved for element 'C' in sheet 'Cadmium' at simulation_images\Cadmium_C_simulation.png
Graph saved for element 'D' in sheet 'Cadmium' at simulation_images\Cadmium_D_simulation.png
Graph saved for element 'E' in sheet 'Cadmium' at simulation_images\Cadmium_E_simulation.png
Graph saved for element 'F' in sheet 'Cadmium' at simulation_images\Cadmium_F_simulation.png
Graph saved for element 'G' in sheet 'Cadmium' at simulation_images\Cadmium_G_simulation.png
Graph saved for element 'H' in sheet 'Cadmium' at simulation_images\Cadmium_H_simulation.png
Graph saved for element 'I' in sheet 'Cadmium' at simulation_images\Cadmium_I_simulation.png
Graph saved for element 'J' in sheet 'Cadmium' at simulation_images\Cadmium_J_simulation.png
Graph saved for element 'K' in sheet 'Cadmium' at simulation_images\Cadmium_K_simulation.png
Graph saved for element 'L' in sheet 'Cadmium' at simulation_images\Ca

...

Graph saved for element 'A' in sheet 'Selenium' at simulation_images\Selenium_A_simulation.png
Graph saved for element 'B' in sheet 'Selenium' at simulation_images\Selenium_B_simulation.png
Graph saved for element 'C' in sheet 'Selenium' at simulation_images\Selenium_C_simulation.png
Graph saved for element 'D' in sheet 'Selenium' at simulation_images\Selenium_D_simulation.png
Graph saved for element 'E' in sheet 'Selenium' at simulation_images\Selenium_E_simulation.png
Graph saved for element 'F' in sheet 'Selenium' at simulation_images\Selenium_F_simulation.png
Graph saved for element 'G' in sheet 'Selenium' at simulation_images\Selenium_G_simulation.png
Graph saved for element 'H' in sheet 'Selenium' at simulation_images\Selenium_H_simulation.png
Graph saved for element 'I' in sheet 'Selenium' at simulation_images\Selenium_I_simulation.png
Graph saved for element 'J' in sheet 'Selenium' at simulation_images\Selenium_J_simulation.png
Graph saved for element 'K' in sheet 'Selenium' at

C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\938653352.py:47: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'B' in sheet 'Vanadium' at simulation_images\Vanadium_B_simulation.png
Graph saved for element 'C' in sheet 'Vanadium' at simulation_images\Vanadium_C_simulation.png
Graph saved for element 'D' in sheet 'Vanadium' at simulation_images\Vanadium_D_simulation.png
Graph saved for element 'E' in sheet 'Vanadium' at simulation_images\Vanadium_E_simulation.png
Graph saved for element 'F' in sheet 'Vanadium' at simulation_images\Vanadium_F_simulation.png
Graph saved for element 'G' in sheet 'Vanadium' at simulation_images\Vanadium_G_simulation.png
Graph saved for element 'H' in sheet 'Vanadium' at simulation_images\Vanadium_H_simulation.png
Graph saved for element 'I' in sheet 'Vanadium' at simulation_images\Vanadium_I_simulation.png
Graph saved for element 'J' in sheet 'Vanadium' at simulation_images\Vanadium_J_simulation.png
Graph saved for element 'K' in sheet 'Vanadium' at simulation_images\Vanadium_K_simulation.png
Graph saved for element 'L' in sheet 'Vanadium' at

C:\Users\nikhi\AppData\Local\Temp\ipykernel_9432\938653352.py:47: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  yearly_change = sheet_df[years].pct_change(axis=1, periods=1, fill_method='pad', limit=None)


Graph saved for element 'B' in sheet 'Zinc' at simulation_images\Zinc_B_simulation.png
Graph saved for element 'C' in sheet 'Zinc' at simulation_images\Zinc_C_simulation.png
Graph saved for element 'D' in sheet 'Zinc' at simulation_images\Zinc_D_simulation.png
Graph saved for element 'E' in sheet 'Zinc' at simulation_images\Zinc_E_simulation.png
Graph saved for element 'F' in sheet 'Zinc' at simulation_images\Zinc_F_simulation.png
Graph saved for element 'G' in sheet 'Zinc' at simulation_images\Zinc_G_simulation.png
Graph saved for element 'H' in sheet 'Zinc' at simulation_images\Zinc_H_simulation.png
Graph saved for element 'I' in sheet 'Zinc' at simulation_images\Zinc_I_simulation.png
Graph saved for element 'J' in sheet 'Zinc' at simulation_images\Zinc_J_simulation.png
Graph saved for element 'K' in sheet 'Zinc' at simulation_images\Zinc_K_simulation.png
Graph saved for element 'L' in sheet 'Zinc' at simulation_images\Zinc_L_simulation.png
Graph saved for element 'M' in sheet 'Zinc'